<a href="https://colab.research.google.com/github/whyclos/time-series/blob/main/twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Домашнее задание №3 по курсу "Анализ временных рядов"

Соколова Диана БМТ1-12М 

In [15]:
import pandas as pd
import plotly.express as px
import numpy as np
import re
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [23]:
cd '/content/gdrive/MyDrive/Colab Notebooks/'

/content/gdrive/MyDrive/Colab Notebooks


In [6]:
#инфо о вакцинациях
df1 = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/vaccinations.csv')
df1.head()

,location,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,total_boosters_per_hundred,daily_vaccinations_per_million
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0
3,Afghanistan,AFG,2021-02-25,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0
4,Afghanistan,AFG,2021-02-26,NaN,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,NaN,34.0


In [8]:
#смотрим только связь дня и количества вакцинаций в день
df_v=df1.groupby(['date'])['daily_vaccinations'].sum().reset_index()
df_v['date'] = pd.to_datetime(df_v['date']).dt.date
df_v.head()

,date,daily_vaccinations
0,2020-12-02,0.0
1,2020-12-03,0.0
2,2020-12-04,0.0
3,2020-12-05,0.0
4,2020-12-06,0.0


In [9]:
fig = px.bar(df_v, x='date', y='daily_vaccinations', labels={'date': 'Date', 'daily_vaccinations': 'Daily vaccinations'})
fig.show()

In [10]:
df2 = pd.read_csv('vaccination_all_tweets.csv')
df2['date'] = pd.to_datetime(df2['date']).dt.date
df2.head()

,id,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,retweets,favorites,is_retweet
0,1340539111971516416,Rachel Roh,"La Crescenta-Montrose, CA",Aggregator of Asian American news; scanning di...,2009-04-08 17:52:46,405,1692,3247,False,2020-12-20,Same folks said daikon paste could treat a cyt...,['PfizerBioNTech'],Twitter for Android,0,0,False
1,1338158543359250433,Albert Fong,"San Francisco, CA","Marketing dude, tech geek, heavy metal & '80s ...",2009-09-21 15:27:30,834,666,178,False,2020-12-13,While the world has been on the wrong side of ...,NaN,Twitter Web App,1,1,False
2,1337858199140118533,eli🇱🇹🇪🇺👌,Your Bed,"heil, hydra 🖐☺",2020-06-25 23:30:28,10,88,155,False,2020-12-12,#coronavirus #SputnikV #AstraZeneca #PfizerBio...,"['coronavirus', 'SputnikV', 'AstraZeneca', 'Pf...",Twitter for Android,0,0,False
3,1337855739918835717,Charles Adler,"Vancouver, BC - Canada","Hosting ""CharlesAdlerTonight"" Global News Radi...",2008-09-10 11:28:53,49165,3933,21853,True,2020-12-12,"Facts are immutable, Senator, even when you're...",NaN,Twitter Web App,446,2129,False
4,1337854064604966912,Citizen News Channel,NaN,Citizen News Channel bringing you an alternati...,2020-04-23 17:58:42,152,580,1473,False,2020-12-12,Explain to me again why we need a vaccine @Bor...,"['whereareallthesickpeople', 'PfizerBioNTech']",Twitter for iPhone,0,0,False


In [12]:
#смотрим только кол-во твитов и даты
timeline = df2.groupby(['date']).count().reset_index()
timeline['count'] = timeline['text']
timeline = timeline[['date', 'count']]
fig = px.bar(timeline, x='date', y='count', labels={'date': 'Date', 'count': 'Tweet Count'})
fig.show()

In [14]:
df3 = timeline.merge(df_v, how='left', on='date')
df3 = df3.dropna()  

Проверка, является ли временной ряд из второй колонки причиной для временного ряда из первой колонки

Нулевая гипотеза: временной ряд X (вторая колонка) не вызывает временной ряд Y (первая колонка)

In [18]:
grangercausalitytests(df3[['count', 'daily_vaccinations']], maxlag = 5 )


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=25.8205 , p=0.0000  , df_denom=231, df_num=1
ssr based chi2 test:   chi2=26.1558 , p=0.0000  , df=1
likelihood ratio test: chi2=24.7945 , p=0.0000  , df=1
parameter F test:         F=25.8205 , p=0.0000  , df_denom=231, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=13.7551 , p=0.0000  , df_denom=228, df_num=2
ssr based chi2 test:   chi2=28.1134 , p=0.0000  , df=2
likelihood ratio test: chi2=26.5425 , p=0.0000  , df=2
parameter F test:         F=13.7551 , p=0.0000  , df_denom=228, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=7.1025  , p=0.0001  , df_denom=225, df_num=3
ssr based chi2 test:   chi2=21.9704 , p=0.0001  , df=3
likelihood ratio test: chi2=20.9914 , p=0.0001  , df=3
parameter F test:         F=7.1025  , p=0.0001  , df_denom=225, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=4.3195  , p=0.0022  

{1: ({'ssr_ftest': (25.8204685445575, 7.729147720740899e-07, 231.0, 1),
   'ssr_chi2test': (26.155799304876428, 3.149497903367016e-07, 1),
   'lrtest': (24.794538043401644, 6.377793646858007e-07, 1),
   'params_ftest': (25.82046854335296, 7.729147725078515e-07, 231.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (13.755073080453613, 2.291096663025654e-06, 228.0, 2),
   'ssr_chi2test': (28.11343883987449, 7.856775110951334e-07, 2),
   'lrtest': (26.54254238027079, 1.7232970885275841e-06, 2),
   'params_ftest': (13.755073081317278, 2.291096661259944e-06, 228.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (7.1024862969835265, 0.0001400444231899132, 225.0, 3),
   'ssr_chi2test': (21.970357612002374, 6.616409903570136e-05, 3),
   'lrtest': (20.99140320467086, 0.00010570984587596594, 3),
   'params_ftest': (7.102486296956151, 0.0001400444231949427, 225.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0

Нулевую гипотезу можно отвергнуть, поэтому количество ежедневной вакцинации является причиной количества твитов.

In [20]:
grangercausalitytests(df3[['daily_vaccinations', 'count',]], maxlag = 5)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0159  , p=0.8998  , df_denom=231, df_num=1
ssr based chi2 test:   chi2=0.0161  , p=0.8991  , df=1
likelihood ratio test: chi2=0.0161  , p=0.8991  , df=1
parameter F test:         F=0.0159  , p=0.8998  , df_denom=231, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=1.3661  , p=0.2572  , df_denom=228, df_num=2
ssr based chi2 test:   chi2=2.7921  , p=0.2476  , df=2
likelihood ratio test: chi2=2.7755  , p=0.2496  , df=2
parameter F test:         F=1.3661  , p=0.2572  , df_denom=228, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=1.5576  , p=0.2006  , df_denom=225, df_num=3
ssr based chi2 test:   chi2=4.8181  , p=0.1856  , df=3
likelihood ratio test: chi2=4.7688  , p=0.1895  , df=3
parameter F test:         F=1.5576  , p=0.2006  , df_denom=225, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=1.7695  , p=0.1360  

{1: ({'ssr_ftest': (0.015879002397229062, 0.8998321000647359, 231.0, 1),
   'ssr_chi2test': (0.01608522320758269, 0.8990769032656225, 1),
   'lrtest': (0.016084670382042532, 0.8990786282912387, 1),
   'params_ftest': (0.015879002397315992, 0.8998321000644944, 231.0, 1.0)},
   array([[0., 1., 0.]])]),
 2: ({'ssr_ftest': (1.3661127208138353, 0.257176627655266, 228.0, 2),
   'ssr_chi2test': (2.792142666224769, 0.24756766680400746, 2),
   'lrtest': (2.775545385791702, 0.24963069010454617, 2),
   'params_ftest': (1.3661127208137398, 0.25717662765529037, 228.0, 2.0)},
   array([[0., 0., 1., 0., 0.],
          [0., 0., 0., 1., 0.]])]),
 3: ({'ssr_ftest': (1.557577212574662, 0.20055051673049742, 225.0, 3),
   'ssr_chi2test': (4.818105510897621, 0.18561127847290598, 3),
   'lrtest': (4.768757100899165, 0.189534402533642, 3),
   'params_ftest': (1.5575772125745608, 0.20055051673052507, 225.0, 3.0)},
   array([[0., 0., 0., 1., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0.],
          [0., 0.

Нулевую гипотезу нельзя отвергнуть, поэтому нельзя однозначно сказать, что количество твитов влияет на количество ежедневной вакцинации.

Для того, чтобы определить, являются ли данные процессы стационарными, воспользуемся тестом Дики-Фуллера, принимая в качестве нулевой гипотезы то, что процесс не является стационарным

In [21]:
df3

,date,count,daily_vaccinations
0,2020-12-12,63,15297.0
1,2020-12-13,98,50750.0
2,2020-12-14,137,139627.0
3,2020-12-15,150,371204.0
4,2020-12-16,144,580596.0
...,...,...,...
230,2021-08-24,860,141618411.0
231,2021-08-25,852,144342936.0
232,2021-08-26,1021,147456968.0
233,2021-08-27,928,148928388.0


In [22]:
test_stat, pvalue, n_lags, _, _, _  = adfuller(
    df3['count'], autolag='AIC'
)
print(f'test_stat = {test_stat}\npvalue = {pvalue}\nn_lags = {n_lags}')

test_stat = -1.662968605801488
pvalue = 0.4502860042813275
n_lags = 13


Т.к. p-значение больше 0.05, то нулевую гипотезу нельзя опровергнуть, значит процесс количества вакцинации в день не является стационарным

**ВЫВОДЫ: **
1. количество ежедневной вакцинации является причиной количества твитов.
2. количество твитов не влияет на количество ежедневной вакцинации.
3. процесс вакцинации не является стационарным